In [43]:
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint 
import os
from glob import glob
from tqdm import tqdm

In [70]:
LABEL_INT_DICT = np.unique(pd.read_csv('human_act_train_vid.csv')['label'])


LABER_STR_DICT = {k:v for v, k in enumerate(LABEL_INT_DICT)}

pprint(LABER_STR_DICT)


{'fall': 0, 'load': 1, 'sit': 2, 'walk': 3}


In [45]:
import math
import tensorflow as tf
from tensorflow import keras

import tensorflow.keras.backend as K
random.seed()
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size,  mode ='train', shuffle = True): # img size 부분도 1280 여기서는 이렇게 고정이 되는 것임1
        assert mode in ['train','valid']

        self.mode = mode
        self.shuffle =shuffle
        self.batch_size = batch_size

        
        self.npy_paths = glob(
            f'human_act_skeleton_npy/{mode}/*.npy'
        )
        random.shuffle(self.npy_paths)

    def __len__(self): # 한 epoch에 몇개가 들어가는 지
        return math.ceil(len(self.npy_paths) / self.batch_size)

    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.npy_paths[strt:fin]
        
        batch_x , batch_y = self.get_data(data)
        # print(K.constant(np.array(batch_x)) ,K.constant( np.array(batch_y)))
        return K.constant(np.array(batch_x)) ,K.constant( np.array(batch_y))

    def get_data(self, data):
        batch_x = []
        batch_y = []

        for npy_path in data:
            # print()
            npy= np.load(npy_path)
            # print(npy_path)
            # 경로 지정하는 게 중요하다
            label = npy_path.split('\\')[-1]. \
                split('_')[0]

            label = LABER_STR_DICT[label]
            
            batch_x.append(npy)
            batch_y.append(label)
        return batch_x, batch_y
    
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.npy_paths)



train_generator = DataGenerator(
    mode = 'train',
    batch_size=32,

    shuffle=True
)
    

valid_generator = DataGenerator(
    mode = 'valid',
    batch_size=32,

    shuffle=True
)

In [64]:

import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import LSTM, Dense,GRU,LeakyReLU
os.environ['CUDA_VISIBLE_DEVICES'] = '1'




MAX_FRAMES = 100 # 여기서는 100개


def build_model():
    #100 frame, 33*4 132
    model = keras.models.Sequential()
    model.add(GRU(64, return_sequences=True, input_shape=(100,132))) 
    model.add(LeakyReLU(alpha=0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(LeakyReLU(alpha=0.1))
    model.add(GRU(256, return_sequences=True))
    model.add(LeakyReLU(alpha=0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(LeakyReLU(alpha=0.1))
    model.add(GRU(64, return_sequences=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(LABEL_INT_DICT), activation='softmax'))
    return model

adam = keras.optimizers.Adam(lr=0.0001)

model = build_model()

model.compile(
    optimizer=adam,
    loss ="sparse_categorical_crossentropy",
    metrics = ['accuracy']
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_10 (GRU)                 (None, 100, 64)           38016     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 100, 64)           0         
_________________________________________________________________
gru_11 (GRU)                 (None, 100, 128)          74496     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 100, 128)          0         
_________________________________________________________________
gru_12 (GRU)                 (None, 100, 256)          296448    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 100, 256)          0         
_________________________________________________________________
gru_13 (GRU)                 (None, 100, 128)         

In [47]:
# import os
# os.mkdir('human_act_skeleton_weights')


In [48]:

#https://m.blog.naver.com/sayney1004/221863897825
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

c:\Users\ESL-SERVER\anaconda3\envs\py37tf22\lib\site-packages\tensorflow\python\client\session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [49]:

i=0
for x, y in train_generator:
    t_x = x.shape
    t_y=y.shape
    print(t_x, t_y ,y)
    i+=1
    if i==100:
        break
    

(32, 100, 132) (32,) tf.Tensor(
[2. 0. 3. 1. 3. 3. 1. 2. 2. 3. 0. 1. 0. 2. 1. 3. 0. 1. 1. 3. 1. 3. 3. 0.
 1. 0. 3. 0. 1. 2. 0. 1.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[0. 2. 3. 3. 3. 3. 3. 1. 3. 0. 1. 2. 2. 3. 2. 0. 2. 2. 0. 0. 3. 2. 2. 3.
 0. 0. 0. 3. 2. 1. 2. 2.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[2. 2. 0. 1. 0. 1. 3. 3. 1. 0. 2. 3. 2. 1. 3. 1. 0. 1. 1. 1. 3. 0. 0. 1.
 1. 2. 0. 3. 1. 0. 1. 1.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[1. 0. 1. 3. 1. 2. 3. 3. 0. 3. 0. 1. 1. 1. 2. 2. 3. 1. 3. 3. 0. 0. 0. 3.
 3. 3. 3. 2. 0. 1. 2. 0.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[1. 3. 3. 3. 1. 0. 0. 1. 1. 1. 0. 1. 3. 3. 0. 0. 2. 1. 3. 3. 3. 2. 3. 0.
 3. 1. 3. 3. 3. 3. 2. 3.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[3. 3. 0. 3. 2. 3. 3. 0. 0. 2. 3. 3. 0. 2. 0. 1. 2. 2. 1. 1. 1. 2. 3. 0.
 1. 0. 2. 2. 0. 1. 1. 1.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[1. 3. 2

In [50]:
i=0
for x, y in valid_generator:
    t_x = x.shape
    t_y=y.shape
    print(t_x,t_y,y)
    i+=1
    if i==100:
        break

(32, 100, 132) (32,) tf.Tensor(
[2. 0. 0. 0. 3. 1. 3. 3. 3. 1. 3. 3. 3. 1. 3. 2. 3. 3. 1. 3. 1. 0. 0. 3.
 2. 3. 3. 1. 0. 1. 3. 1.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[0. 0. 3. 0. 0. 3. 0. 1. 3. 3. 3. 3. 0. 3. 1. 3. 1. 2. 0. 3. 3. 0. 2. 3.
 1. 2. 1. 3. 1. 2. 3. 3.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[1. 3. 1. 1. 1. 2. 1. 1. 2. 2. 2. 3. 3. 3. 1. 0. 0. 3. 0. 2. 3. 3. 2. 1.
 3. 2. 3. 1. 3. 2. 2. 2.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[0. 3. 3. 3. 3. 3. 1. 2. 2. 1. 3. 0. 1. 1. 1. 2. 3. 1. 3. 3. 2. 1. 1. 2.
 0. 3. 3. 2. 3. 1. 3. 3.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[2. 2. 1. 0. 2. 3. 3. 1. 3. 3. 0. 1. 1. 1. 1. 1. 3. 2. 0. 0. 0. 1. 3. 1.
 0. 1. 1. 2. 3. 2. 1. 3.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[3. 3. 1. 3. 1. 3. 1. 0. 3. 1. 1. 1. 3. 0. 3. 2. 3. 3. 2. 1. 3. 0. 2. 3.
 3. 2. 2. 0. 0. 3. 2. 2.], shape=(32,), dtype=float32)
(32, 100, 132) (32,) tf.Tensor(
[3. 3. 2

In [51]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 100, 132) <dtype: 'float32'>
(None, 4) <dtype: 'float32'>
gru_5 (None, 100, 132) float32
gru_6 (None, 100, 64) float32
gru_7 (None, 100, 128) float32
gru_8 (None, 100, 256) float32
gru_9 (None, 100, 128) float32
dense_9 (None, 64) float32
dense_10 (None, 64) float32
dense_11 (None, 32) float32


[None, None, None, None, None, None, None, None]

In [72]:

filepath = 'human_act_skeleton_weights/human_skeleton_act-{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='min'

    
)


history = model.fit(

    train_generator,
    validation_data= valid_generator,
    epochs=100,
    verbose=1,
    callbacks=[
        model_checkpoint
    ],  
      
)


Epoch 1/100
27/27 [==============================] - ETA: 0s - loss: 0.1569 - accuracy: 0.9426
Epoch 00001: val_loss improved from inf to 0.28499, saving model to human_act_skeleton_weights/human_skeleton_act-01-0.90.hdf5
27/27 [==============================] - 19s 693ms/step - loss: 0.1569 - accuracy: 0.9426 - val_loss: 0.2850 - val_accuracy: 0.9007
Epoch 2/100
27/27 [==============================] - ETA: 0s - loss: 0.1844 - accuracy: 0.9286
Epoch 00002: val_loss did not improve from 0.28499
27/27 [==============================] - 18s 653ms/step - loss: 0.1844 - accuracy: 0.9286 - val_loss: 0.3977 - val_accuracy: 0.8742
Epoch 3/100
27/27 [==============================] - ETA: 0s - loss: 0.2005 - accuracy: 0.9262
Epoch 00003: val_loss did not improve from 0.28499
27/27 [==============================] - 17s 646ms/step - loss: 0.2005 - accuracy: 0.9262 - val_loss: 0.3235 - val_accuracy: 0.8576
Epoch 4/100
27/27 [==============================] - ETA: 0s - loss: 0.1830 - accuracy: 0.

In [66]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose


In [67]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)  
    return pose

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR

    return image, results

def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    
                            

In [73]:
import tensorflow as tf

model = tf.keras.models.load_model('human_act_skeleton_weights\human_skeleton_act-99-0.92.hdf5')

# 1. New detection variables
sequence = []

threshold = 0.8
# path='6-3_002-C04.mp4'
path=0
cap = cv2.VideoCapture(path)
# Set mediapipe model 
with mp_pose.Pose( min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, pose)
        # print(results)
        
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-100:]


        text='no'
        if len(sequence) == 100:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            # text= LABEL_INT_DICT[np.argmax(res)]
            text= str(np.argmax(res))

        print(text)
        #3. Viz logic
        draw_styled_landmarks(image,results)
            
        # cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' val =  '+(text), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2